# AI Car Detection - Dashcam Video Simulation

This notebook demonstrates object detection on a dashcam video, focusing on detecting cars and vehicles using a pre-trained YOLO model.

## 1. Install and Import Required Libraries

In [ ]:
# Install required packages (using pre-built wheels to avoid compilation)
%pip install --upgrade pip
%pip install numpy --upgrade --only-binary :all:
%pip install opencv-python-headless matplotlib IPython requests --only-binary :all:

In [ ]:
# Import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, display, Video
from base64 import b64encode
import tarfile
import os

print("Libraries imported successfully!")

## 2. Download and Load Pre-trained YOLO Model

We'll use YOLOv3 (You Only Look Once) which is excellent for real-time object detection including cars.

In [ ]:
# Download model files if they don't exist
import urllib.request

def download_file(url, destination, description="file"):
    """Download a file if it doesn't already exist."""
    if os.path.exists(destination):
        print(f"✓ {description} already exists")
        return True
    
    print(f"Downloading {description}...")
    os.makedirs(os.path.dirname(destination), exist_ok=True)
    
    try:
        def show_progress(block_num, block_size, total_size):
            downloaded = block_num * block_size
            if total_size > 0:
                percent = min(downloaded * 100.0 / total_size, 100.0)
                mb_downloaded = downloaded / (1024 * 1024)
                mb_total = total_size / (1024 * 1024)
                print(f"\rProgress: {percent:.1f}% ({mb_downloaded:.1f}/{mb_total:.1f} MB)", end='')
        
        urllib.request.urlretrieve(url, destination, show_progress)
        print(f"\n✓ Downloaded {description}")
        return True
    except Exception as e:
        print(f"\n✗ Error: {e}")
        return False

# Download YOLOv3 model files
print("Checking model files...\n")

files = [
    ("https://pjreddie.com/media/files/yolov3.weights", 
     "models/yolov3.weights", "YOLOv3 weights (237 MB)"),
    ("https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg", 
     "models/yolov3.cfg", "YOLOv3 config"),
    ("https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names", 
     "models/coco.names", "COCO class names")
]

for url, dest, desc in files:
    download_file(url, dest, desc)

print("\n" + "="*60)

In [ ]:
# Load YOLO model
print("\nLoading YOLO model...")
net = cv2.dnn.readNet("models/yolov3.weights", "models/yolov3.cfg")

# Load class names
with open("models/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Get output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

print(f"✓ YOLO model loaded successfully!")
print(f"Total classes: {len(classes)}")
print(f"Vehicle-related classes: car, truck, bus, motorbike")

## 3. Load Dashcam Video

In [ ]:
# Load the dashcam video
video_path = "videos/dash_cam.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video loaded: {video_path}")
print(f"Resolution: {width}x{height}")
print(f"FPS: {fps}")
print(f"Total frames: {total_frames}")
print(f"Duration: {total_frames/fps:.2f} seconds")

cap.release()

## 4. Define Car Detection Function

In [ ]:
def detect_objects(frame):
    """Detect objects in a frame using YOLO"""
    height, width = frame.shape[:2]
    
    # Prepare image for YOLO
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)
    
    # Process detections
    boxes = []
    confidences = []
    class_ids = []
    
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            # Filter for vehicles with confidence > 0.5
            if confidence > 0.5 and classes[class_id] in ['car', 'truck', 'bus', 'motorbike']:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w/2)
                y = int(center_y - h/2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    # Apply non-maximum suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    # Draw boxes
    detected_count = 0
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
            color = (0, 255, 0)  # Green for vehicles
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            detected_count += 1
    
    return frame, detected_count

print("Detection function ready!")

## 5. Process Video and Detect Cars

In [ ]:
# Process video
cap = cv2.VideoCapture(video_path)

# Output video writer
output_path = "videos/detected_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
total_detections = 0

print("Processing video frames...")
print("This may take a few minutes depending on video length...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detect objects in frame
    processed_frame, detections = detect_objects(frame)
    total_detections += detections
    
    # Write processed frame
    out.write(processed_frame)
    
    frame_count += 1
    if frame_count % 30 == 0:  # Progress update every 30 frames
        print(f"Processed {frame_count}/{total_frames} frames...")

cap.release()
out.release()

print(f"\n✓ Processing complete!")
print(f"Total frames processed: {frame_count}")
print(f"Average vehicles per frame: {total_detections/frame_count:.2f}")
print(f"Output saved to: {output_path}")

## 6. Display Sample Detection Results

Let's display a few sample frames to see the detection results.

In [ ]:
# Display sample frames from processed video
cap = cv2.VideoCapture(output_path)

# Get frames at different timestamps
sample_positions = [0.2, 0.4, 0.6, 0.8]  # 20%, 40%, 60%, 80% through video
sample_frames = []

for pos in sample_positions:
    cap.set(cv2.CAP_PROP_POS_FRAMES, int(total_frames * pos))
    ret, frame = cap.read()
    if ret:
        # Convert BGR to RGB for matplotlib
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        sample_frames.append(frame_rgb)

cap.release()

# Display frames
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Car Detection Results - Sample Frames', fontsize=16, fontweight='bold')

for idx, (ax, frame) in enumerate(zip(axes.flat, sample_frames)):
    ax.imshow(frame)
    ax.set_title(f'Frame at {int(sample_positions[idx]*100)}%', fontsize=12)
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Sample frames displayed above with detected vehicles highlighted in green boxes.")